Reference

* Awesome Kernel, thanks @martinpiotte : https://www.kaggle.com/martinpiotte/bounding-box-model
* cropping.model: https://www.kaggle.com/martinpiotte/bounding-box-model/output
* Data: https://www.kaggle.com/c/humpback-whale-identification/data
* Bounding Boxes generation: https://www.kaggle.com/suicaokhoailang/generating-whale-bounding-boxes

In [ ]:
!apt-get install zip
import pandas as pd
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.models as models

from os import listdir
from os.path import isfile, join

from PIL import Image

In [ ]:
TRAIN_CSV = '../input/humpback-whale-identification/train.csv'
TRAIN = '../input/humpback-whale-identification/train'
TEST = '../input/humpback-whale-identification/test'
BBOX = '../input/generating-whale-bounding-boxes-ba9082/bounding_boxes.csv'

TRAIN_CROPPED = 'output/humpback-whale-identification-cropped/train'
TEST_CROPPED  = 'output/humpback-whale-identification-cropped/test'

In [ ]:
train_csv = pd.read_csv(TRAIN_CSV)
print(train_csv[:10])

In [ ]:
bbox = pd.read_csv(BBOX)
print(bbox[:10])

In [ ]:
!mkdir -p 'output/humpback-whale-identification-cropped/train'

train_total = train_csv.shape[0]
train_count = 0

print_every = 50

for row in train_csv.iterrows():
    train_count += 1
    res = row[1]['Image']
    path = os.path.join(TRAIN, res)
    dst = os.path.join(TRAIN_CROPPED, res)
    img = Image.open(path).convert('RGB')
    bbox_row = bbox[bbox['Image'] == res].iloc[0]
    x0, y0, x1, y1 = bbox_row['x0'], bbox_row['y0'], bbox_row['x1'], bbox_row['y1']
    img = img.crop((x0, y0, x1, y1))
    img.save(dst, 'jpeg')
    
    if train_count % print_every == 0:
        print("Cropping Training Set -> {:3.2f}%".format((train_count/train_total) * 100), end="\r")

print("Cropping Training Set -> 100%", end="\r")

In [ ]:
!zip -rm train_cropped.zip output/humpback-whale-identification-cropped/train

In [ ]:
!mkdir -p 'output/humpback-whale-identification-cropped/test'

test_count = 0

print_every = 50

files = [f for f in listdir(TEST) if isfile(join(TEST, f))]
test_total = len(files)

for res in files:
    test_count += 1
    path = os.path.join(TEST, res)
    dst = os.path.join(TEST_CROPPED, res)
    img = Image.open(path).convert('RGB')
    bbox_row = bbox[bbox['Image'] == res].iloc[0]
    x0, y0, x1, y1 = bbox_row['x0'], bbox_row['y0'], bbox_row['x1'], bbox_row['y1']
    img = img.crop((x0, y0, x1, y1))
    img.save(dst, 'jpeg')
    
    if test_count % print_every == 0:
        print("Cropping Testing Set -> {:3.2f}%".format((test_count/test_total) * 100), end="\r")

print("Cropping Testing Set -> 100%", end="\r")

In [ ]:
!zip -rm test_cropped.zip output/humpback-whale-identification-cropped/test